In [1]:
import numpy as np
import pandas as pd
import sqlite3

## Download Dataset

In [2]:
spotify_songs = pd.read_csv('https://raw.githubusercontent.com/rfordatascience/tidytuesday/master/data/2020/2020-01-21/spotify_songs.csv')
spotify_songs = spotify_songs.drop_duplicates()
spotify_songs.head()

,track_id,track_name,track_artist,track_popularity,track_album_id,track_album_name,track_album_release_date,playlist_name,playlist_id,playlist_genre,...,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms
0,6f807x0ima9a1j3VPbc7VN,I Don't Care (with Justin Bieber) - Loud Luxur...,Ed Sheeran,66,2oCs0DGTsRO98Gh5ZSl2Cx,I Don't Care (with Justin Bieber) [Loud Luxury...,2019-06-14,Pop Remix,37i9dQZF1DXcZDD7cfEKhW,pop,...,6,-2.634,1,0.0583,0.1020,0.000000,0.0653,0.518,122.036,194754
1,0r7CVbZTWZgbTCYdfa2P31,Memories - Dillon Francis Remix,Maroon 5,67,63rPSO264uRjW1X5E6cWv6,Memories (Dillon Francis Remix),2019-12-13,Pop Remix,37i9dQZF1DXcZDD7cfEKhW,pop,...,11,-4.969,1,0.0373,0.0724,0.004210,0.3570,0.693,99.972,162600
2,1z1Hg7Vb0AhHDiEmnDE79l,All the Time - Don Diablo Remix,Zara Larsson,70,1HoSmj2eLcsrR0vE9gThr4,All the Time (Don Diablo Remix),2019-07-05,Pop Remix,37i9dQZF1DXcZDD7cfEKhW,pop,...,1,-3.432,0,0.0742,0.0794,0.000023,0.1100,0.613,124.008,176616
3,75FpbthrwQmzHlBJLuGdC7,Call You Mine - Keanu Silva Remix,The Chainsmokers,60,1nqYsOef1yKKuGOVchbsk6,Call You Mine - The Remixes,2019-07-19,Pop Remix,37i9dQZF1DXcZDD7cfEKhW,pop,...,7,-3.778,1,0.1020,0.0287,0.000009,0.2040,0.277,121.956,169093
4,1e8PAfcKUYoKkxPhrHqw4x,Someone You Loved - Future Humans Remix,Lewis Capaldi,69,7m7vv9wlQ4i0LFuJiE2zsQ,Someone You Loved (Future Humans Remix),2019-03-05,Pop Remix,37i9dQZF1DXcZDD7cfEKhW,pop,...,1,-4.672,1,0.0359,0.0803,0.000000,0.0833,0.725,123.976,189052


In [3]:
spotify_songs.shape

(32833, 23)

## Create a SQLite3 schema

### Break Table

Based on database normalization rule as well as personal preference, the Spotify Song dataset will be divided into $6$ parts. The result is shown as below.

|Table|Variables|Primary Key|Shape|
|:---:|:-------:|:---------:|:---:|
|**song**|`track_id` `track_name` `track_artist` `track_popularity` `track_album_id`|`track_id`|(28356, 5)|
|**album**|`track_album_id` `track_album_name` `track_album_release_date`|`track_album_id`|(22545, 3)|
|**playlist**|`playlist_id` `playlist_name`|`playlist_id`|(471, 2)|
|**genre**|`playlist_subgenre` `playlist_genre`|`playlist_subgenre`|(24, 2)|
|**feature**|`track_id` `danceability` `energy` `key` `loudness` `mode` `speechiness` `acousticness` `instrumentalness` `liveness` `valence` `tempo` `duration_ms`|`track_id`|(28356, 13)|
|**connection**|`track_id` `playlist_id` `playlist_subgenre`|`track_id` `playlist_id` `playlist_subgenre`|(32833, 3)|

In [4]:
song = spotify_songs[['track_id', 'track_name', 'track_artist', 'track_popularity', 'track_album_id']].drop_duplicates()
album = spotify_songs[['track_album_id', 'track_album_name', 'track_album_release_date']].drop_duplicates()
playlist = spotify_songs[['playlist_id', 'playlist_name']].drop_duplicates()
genre = spotify_songs[['playlist_subgenre', 'playlist_genre']].drop_duplicates()
feature = spotify_songs[['track_id', 'danceability', 'energy', 'key', 'loudness', 'mode', 
                         'speechiness', 'acousticness', 'instrumentalness', 'liveness', 
                         'valence', 'tempo','duration_ms']].drop_duplicates()
feature = round(feature,4)
connection = spotify_songs[['track_id', 'playlist_id', 'playlist_subgenre']].drop_duplicates()

In [5]:
song.name = 'song'
album.name = 'album'
playlist.name = 'playlist'
genre.name = 'genre'
feature.name = 'feature'
connection.name = 'connection'

In [6]:
print("Shape of table 'song':", song.shape)
print("Shape of table 'album':", album.shape)
print("Shape of table 'playlist':", playlist.shape)
print("Shape of table 'genre':", genre.shape)
print("Shape of table 'feature':", feature.shape)
print("Shape of table 'connection':", connection.shape)

Shape of table 'song': (28356, 5)
Shape of table 'album': (22545, 3)
Shape of table 'playlist': (471, 2)
Shape of table 'genre': (24, 2)
Shape of table 'feature': (28356, 13)
Shape of table 'connection': (32833, 3)


### Check Primary Key

In [7]:
def check(table, primary_key):
    """
    This function is to check the validity of primary key.
    """
    if np.any(table[primary_key].isnull()):
        print('Mistake: the primary key of {} contains NULL value!'.format(table.name))
    if table[primary_key].drop_duplicates().shape[0] != table.shape[0]:
        print('Mistake: the primary key of {} contains REPETITIVE value!'.format(table.name))

In [8]:
check(song, 'track_id')
check(album, 'track_album_id')
check(playlist, 'playlist_id')
check(genre, 'playlist_subgenre')
check(feature, 'track_id')
check(connection, ['track_id', 'playlist_id', 'playlist_subgenre'])

### Save as Sqlite3 form

In [9]:
con = sqlite3.connect("Data/spotify.db")
song.to_sql('song', con, if_exists = 'replace', index = False)
album.to_sql('album', con, if_exists = 'replace', index = False)
playlist.to_sql('playlist', con, if_exists = 'replace', index = False)
genre.to_sql('genre', con, if_exists = 'replace', index = False)
feature.to_sql('feature', con, if_exists = 'replace', index = False)
connection.to_sql('connection', con, if_exists = 'replace', index = False)

In [10]:
%load_ext sql

In [11]:
%sql sqlite:///Data/spotify.db

'Connected: @Data/spotify.db'

In [12]:
%%sql

SELECT *
FROM sqlite_master
WHERE type = 'table';

 * sqlite:///Data/spotify.db
Done.


type,name,tbl_name,rootpage,sql
table,song,song,2,"CREATE TABLE ""song"" (""track_id"" TEXT, ""track_name"" TEXT, ""track_artist"" TEXT, ""track_popularity"" INTEGER, ""track_album_id"" TEXT)"
table,album,album,593,"CREATE TABLE ""album"" (""track_album_id"" TEXT, ""track_album_name"" TEXT, ""track_album_release_date"" TEXT)"
table,playlist,playlist,919,"CREATE TABLE ""playlist"" (""playlist_id"" TEXT, ""playlist_name"" TEXT)"
table,genre,genre,927,"CREATE TABLE ""genre"" (""playlist_subgenre"" TEXT, ""playlist_genre"" TEXT)"
table,feature,feature,928,"CREATE TABLE ""feature"" (""track_id"" TEXT, ""danceability"" REAL, ""energy"" REAL, ""key"" INTEGER, ""loudness"" REAL, ""mode"" INTEGER, ""speechiness"" REAL, ""acousticness"" REAL, ""instrumentalness"" REAL, ""liveness"" REAL, ""valence"" REAL, ""tempo"" REAL, ""duration_ms"" INTEGER)"
table,connection,connection,1724,"CREATE TABLE ""connection"" (""track_id"" TEXT, ""playlist_id"" TEXT, ""playlist_subgenre"" TEXT)"


## Exploration

This part is to find the names of all playlists that contain instrumentals. And we take the first 10 playlists as the output ordered by the number of songs within each playlist.

In [13]:
%%sql

SELECT playlist_name AS 'playlist name', COUNT(DISTINCT feature.track_id) AS 'Number of Songs'
FROM feature INNER JOIN connection
ON feature.track_id = connection.track_id
INNER JOIN playlist
ON connection.playlist_id = playlist.playlist_id
WHERE instrumentalness > 0.5
GROUP BY playlist_name
ORDER BY COUNT(DISTINCT feature.track_id) DESC
LIMIT 10;

 * sqlite:///Data/spotify.db
Done.


playlist name,Number of Songs
Lush Lofi,91
Underground Party | Hypnotic | Minimal | Acid | Big Room | Tech | Liquid,88
Jazz Vibes,81
House Electro 2019,81
Lo-Fi Beats,78
Lofi Hip-Hop,75
Selected House,60
💊ELECTRO-HOUSE-TECH💊,58
Sunny Beats,54
Tropical Vibes,52


In [14]:
%%sql

SELECT COUNT(DISTINCT playlist_name) AS 'total number of playlist'
FROM feature INNER JOIN connection
ON feature.track_id = connection.track_id
INNER JOIN playlist
ON connection.playlist_id = playlist.playlist_id
WHERE instrumentalness > 0.5;

 * sqlite:///Data/spotify.db
Done.


total number of playlist
256
